<a href="https://colab.research.google.com/github/MarcosP7635/Energy/blob/main/JupyterNotebooks/solar_icf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import subprocess
running_on_colab = not "JupyterNotebooks" in subprocess.os.getcwd()
if running_on_colab:
    running_on_colab = True
    parent_dir, export_data_dir, import_data_dir = "/content/", "/content/", "/content/"
else:
    parent_dir = subprocess.os.getcwd().split("JupyterNotebooks")[0]
    sep_char = parent_dir.split("Energy")[-1]
    import_data_dir =  parent_dir + "ImportedData" + sep_char 
    export_data_dir =  parent_dir + "ExportedData" + sep_char
parent_dir, import_data_dir, export_data_dir

('c:\\Users\\engin\\Documents\\GitHub\\Energy\\',
 'c:\\Users\\engin\\Documents\\GitHub\\Energy\\ImportedData\\',
 'c:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\')

https://docs.astropy.org/en/stable/units/index.html

https://docs.astropy.org/en/stable/constants/index.html

https://docs.sympy.org/latest/

In [31]:
from astropy import units as units
from astropy import constants as const
if running_on_colab:
    !pip install SymPy
import sympy
from sympy import Sum, symbols, Indexed, lambdify, integrate
from sympy import integrate, solve
from sympy.abc import *
import numpy as np
import pandas as pd

Consider Sun light focused onto a pellet of pure deuterium. What conditions are necessary to produce net energy?
The energy produced by the fusion of deuterium, $E_{prod}$ is

In [32]:
E_produced = Sum(Indexed('E',r) * Indexed('N',r) * Indexed('nu', r), (r,1,R))
E_produced

Sum(E[r]*N[r]*nu[r], (r, 1, R))

In [33]:
if running_on_colab:
  from google_drive_downloader import GoogleDriveDownloader as gdd
  def download(url):
    file_id = url[url.index("/d/") + 3: -1*len("/view?usp=sharing")]
    gdd.download_file_from_google_drive(
    file_id, "/content/x4_output.txt", False)
  download("https://drive.google.com/file/d/1EqAhmeIMu1am2pZnUGC3ZSUpl8FbYOLH/view?usp=sharing")
  file_path = "/content/x4_output.txt"
else:
  file_path = import_data_dir + "x4_ddn.txt"
print(file_path)

c:\Users\engin\Documents\GitHub\Energy\ImportedData\x4_ddn.txt


In [34]:
#import text file as string
with open(file_path, 'r') as f:
    data = f.read().split('\n')
data

['REQUEST        8305001   20221213      70719   20221128          3    0  0    0',
 'ENTRY            A0409   20120925   20130502   20130409       A078A0409000    1',
 'SUBENT        A0409001   20120925   20130502   20130409       A078A0409001    1',
 'BIB                  8         25                                 A0409001    2',
 'TITLE      Low-energy fusion cross-sections of d+d and d+He3      A0409001    3',
 '           reactions.                                             A0409001    4',
 'AUTHOR     (A.Krauss, H.W.Becker, H.P.Trautvetter, C.Rolfs,       A0409001    5',
 '           K.Brand)                                               A0409001    6',
 'REFERENCE  (J,NP/A,465,150,1987)                                  A0409001    7',
 'INSTITUTE  (2GERMST)                                              A0409001    8',
 '           (2GERBOC)                                              A0409001    9',
 'HISTORY    (19890505C)                                            A0409001

The following are the number of columns and the number of rows in the data from each experiment, and where the data from each experiment is found in the file.

In [35]:
def is_data_type(input, type):
    try:
        if type == 'integer':
            int(input)
        elif type == 'float':
            float(input)
        elif type == 'string':
            str(input)
        return True
    except ValueError:
        return False
def filter_x4_output(file_as_array, search_term, requested_data_type, df_width, **kwargs):
    data = file_as_array
    data_rows = [row for row, content in enumerate(data)
                if search_term in content]
    arrays, lines = [], []
    for row in data_rows:
        arr = [x for x in data[row].split(' ') if x!=''][1:df_width+1]
        if all([is_data_type(element, requested_data_type) 
                for element in arr[0:df_width]]):
            if requested_data_type == 'integer':
                arrays.append(np.array(arr).astype(int))
            elif requested_data_type == 'float':
                arrays.append(np.array(arr).astype(float))
            elif requested_data_type == 'string':
                arrays.append(np.array(arr).astype(str))
            lines.append(row)
    if len(kwargs) > 0:
        df = pd.DataFrame(arrays, columns = kwargs['colnames'])
    else:
        df = pd.DataFrame(arrays)
    df["Line in File"] = lines
    return df
df = filter_x4_output(data, 'DATA ', 'integer', 2, 
        colnames=["Columns in Dataset", "Rows in Dataset"])
df

,Columns in Dataset,Rows in Dataset,Line in File
0,3,10,44
1,3,21,72
2,5,8,168
3,3,8,194
4,6,14,263
5,6,18,296
6,3,5,379
7,2,4,413
8,3,9,458
9,3,5,496


Each dataset starts 2 lines below wherever indicated in the above dataframe. 

In [72]:
starting_lines = [x for x in df["Line in File"] + 1]
ending_lines = [x for x in starting_lines + df["Rows in Dataset"] + 2]
list_of_datasets = [data[starting_line:ending_lines[i]]
                    for i, starting_line in enumerate(starting_lines)]
list_of_datasets

[['EN-CM      DATA       DATA-ERR                                    A0409003   14',
  'KEV        B*KEV      B*KEV                                       A0409003   15',
  '       6.63       51.7        5.3                                 A0409003   16',
  '       9.94       51.4        2.0                                 A0409003   17',
  '      14.91       53.5        1.8                                 A0409003   18',
  '      19.88       54.6        1.4                                 A0409003   19',
  '      22.36       56.5        1.1                                 A0409003   20',
  '      29.82       58.2        1.0                                 A0409003   21',
  '      34.78       59.4        1.0                                 A0409003   22',
  '      39.79       58.9        0.8                                 A0409003   23',
  '      44.72       60.9        1.2                                 A0409003   24',
  '      49.67       59.3        0.8                             

In [71]:
list_of_datasets[2]

['KEV        B*KEV      B*KEV      B*KEV      B*KEV                 A0726002   26',
 '       2.35       50.5       88.3                                 A0726002   27',
 '       3.29       67.3       14.0                                 A0726002   28',
 '       3.92       61.2       10.5                                 A0726002   29',
 '       4.68                             50.0        9.0           A0726002   30',
 '       4.70       51.4        8.7                                 A0726002   31',
 '       5.09                             55.0       16.0           A0726002   32',
 '       5.46       54.0        8.7                                 A0726002   33',
 '       6.12       58.2        9.3                                 A0726002   34']

In [70]:
def convert_dataset(dataset):
    list_form = [[x for x in dataset[i].split(" ") if x != ''][:-2]
                        for i in range(len(dataset))]
    output = pd.DataFrame(list_form[1:])#, columns = )
    print(list(set(list_form[0])), output.shape[1])
    return output
convert_dataset(list_of_datasets[2])

['KEV', 'B*KEV'] 3


,0,1,2
0,2.35,50.5,88.3
1,3.29,67.3,14.0
2,3.92,61.2,10.5
3,4.68,50.0,9.0
4,4.70,51.4,8.7
5,5.09,55.0,16.0
6,5.46,54.0,8.7
7,6.12,58.2,9.3
